# Analysis of signle-cell Ca<sup>2+</sup> pathway model with parameters sampled from NUTS

In [1]:
import os.path
import json

import numpy as np

import calcium_models
from stan_helpers import StanSessionAnalyzer, load_trajectories

## Load sampled parameters and Ca<sup>2+</sup> trajectory

In [2]:
stan_run = 'cell-chain-example'
cell_id = 5106

# load metadata
with open('stan_run_meta.json', 'r') as f:
    stan_run_meta = json.load(f)

result_dir = os.path.join(stan_run_meta[stan_run]['output_dir'], 'samples',
                          f'cell-{cell_id:04d}')

In [3]:
# filter parameters
param_mask = stan_run_meta[stan_run]['param_mask']
param_names = [calcium_models.param_names[i + 1]
               for i, mask in enumerate(param_mask) if mask == '1']
param_names = ['sigma'] + param_names

In [4]:
# get Calcium trajectory
t0 = 200
y, y0_ca, ts = load_trajectories(t0, filter_type='moving_average',
    moving_average_window=20, downsample_offset=300)
y = y[cell_id, :]
y0_ca = y0_ca[cell_id]
y0 = np.array([0, 0, 0.7, y0_ca])
y_ref = [None, None, None, y]

In [5]:
# get ODE function
calcium_ode = getattr(
    calcium_models, f'calcium_ode_{stan_run_meta[stan_run]["ode_variant"]}')

var_names = ['PLC', 'IP3', 'h', 'Ca']

In [6]:
# initialize analyzer
analyzer = StanSessionAnalyzer(result_dir, sample_source='arvis_inf_data',
                               param_names=param_names)

## Perform analyses
The code below generates figures saved in `./cell_chain_example/samples/cell-5106`. Generated figures include:
- Trajectories of all variables simulated from sampled parameters
- Trace plot of all parameters
- Violin plots of all parameters
- Pairs plots of all pairs of parameters
- Log posterior likelihood during sampling

In [7]:
_ = analyzer.simulate_chains(calcium_ode, 0, ts, y0, y_ref=y_ref,
                             show_progress=True, var_names=var_names)
analyzer.plot_parameters()
analyzer.plot_log_posteriors()

Simulating trajectories from chain 0...


100%|██████████| 1000/1000 [00:28<00:00, 34.97it/s]


Simulating trajectories from chain 1...


100%|██████████| 1000/1000 [00:28<00:00, 34.73it/s]


Simulating trajectories from chain 2...


100%|██████████| 1000/1000 [00:28<00:00, 34.82it/s]


Simulating trajectories from chain 3...


100%|██████████| 1000/1000 [00:28<00:00, 35.10it/s]


Making trace plot for chain 0...
Making violin plot for chain 0...
Making pairs plot for chain 0...
Making trace plot for chain 1...
Making violin plot for chain 1...
Making pairs plot for chain 1...
Making trace plot for chain 2...
Making violin plot for chain 2...
Making pairs plot for chain 2...
Making trace plot for chain 3...
Making violin plot for chain 3...
Making pairs plot for chain 3...
Plotting log posterior likelihoods of all chains...


<Figure size 432x288 with 0 Axes>